In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn import svm


Read the concatenated CSV

In [2]:
path = "../preprocessing/concat.csv"
df = pd.read_csv(path)

/tmp/ipykernel_20766/2773897866.py:2: DtypeWarning: Columns (721) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


In [3]:
df

,Unnamed: 0,frame,face_id,timestamp,confidence,success,gaze_0_x,gaze_0_y,gaze_0_z,gaze_1_x,...,AU45_c,video_id,file,mix,emotion_1,vocalization,intensity_level,emotion_2,proportions,situation
0,0,1,0,0.00,0.98,1,-0.062394,0.301228,-0.951509,-0.192912,...,1.0,A220,A220_reg_p_2.csv,False,reg,p,2,NaN,NaN,NaN
1,1,2,0,0.02,0.98,1,-0.052638,0.300214,-0.952418,-0.205766,...,0.0,A220,A220_reg_p_2.csv,False,reg,p,2,NaN,NaN,NaN
2,2,3,0,0.04,0.98,1,-0.050868,0.293366,-0.954646,-0.208241,...,0.0,A220,A220_reg_p_2.csv,False,reg,p,2,NaN,NaN,NaN
3,3,4,0,0.06,0.98,1,-0.053854,0.298332,-0.952942,-0.204404,...,0.0,A220,A220_reg_p_2.csv,False,reg,p,2,NaN,NaN,NaN
4,4,5,0,0.08,0.98,1,-0.049128,0.298474,-0.953153,-0.201120,...,0.0,A220,A220_reg_p_2.csv,False,reg,p,2,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110583,305,306,0,6.10,0.98,1,0.085220,0.297743,-0.950835,-0.092437,...,0.0,A220,A220_anx_v_3.csv,False,anx,v,3,NaN,NaN,NaN
110584,306,307,0,6.12,0.98,1,0.081289,0.296204,-0.951659,-0.078804,...,0.0,A220,A220_anx_v_3.csv,False,anx,v,3,NaN,NaN,NaN
110585,307,308,0,6.14,0.98,1,0.079877,0.291925,-0.953100,-0.080352,...,0.0,A220,A220_anx_v_3.csv,False,anx,v,3,NaN,NaN,NaN
110586,308,309,0,6.16,0.93,1,0.083870,0.294762,-0.951883,-0.083980,...,0.0,A220,A220_anx_v_3.csv,False,anx,v,3,NaN,NaN,NaN


Drop mized emotions, get rid of all columns except action units, emotion and filename

In [4]:
df_tmp = df[df['mix'] == False]
df_tmp = df_tmp.loc[:, df_tmp.columns.str.startswith(('AU', "gaze", "pose", "file"))]

Group by filename, calculate mean value

In [5]:
df_tmp = df_tmp.groupby('file').mean()

Create metadata dataframe

In [6]:
metadata_df = df[['file','emotion_1']].drop_duplicates()

Merge metadata (emotion) with action units means and filename

In [7]:
df_tmp = pd.merge(df_tmp, metadata_df, on='file')

In [8]:
df_tmp

,file,gaze_0_x,gaze_0_y,gaze_0_z,gaze_1_x,gaze_1_y,gaze_1_z,gaze_angle_x,gaze_angle_y,pose_Tx,...,AU14_c,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c,emotion_1
0,A220_adm_p_1.csv,0.071776,0.313607,-0.942788,-0.159825,0.321515,-0.928751,-0.046478,0.327467,12.504396,...,0.956044,0.000000,0.307692,0.192308,0.016484,0.219780,0.093407,0.000000,0.247253,adm
1,A220_adm_p_2.csv,0.040902,0.318619,-0.944087,-0.202204,0.334531,-0.915849,-0.086722,0.337805,34.313278,...,0.970954,0.124481,0.307054,0.182573,0.000000,0.307054,0.024896,0.000000,0.219917,adm
2,A220_adm_p_3.csv,0.027176,0.333384,-0.939375,-0.186133,0.333285,-0.920742,-0.084796,0.344277,42.506383,...,0.982979,0.038298,0.297872,0.123404,0.000000,0.110638,0.200000,0.000000,0.289362,adm
3,A220_adm_p_4.csv,0.046896,0.284979,-0.955106,-0.181577,0.271767,-0.942284,-0.070551,0.285456,35.406084,...,0.984791,0.026616,0.266160,0.000000,0.000000,0.015209,0.019011,0.000000,0.209125,adm
4,A220_adm_v_1.csv,0.086057,0.290664,-0.950133,-0.106759,0.318649,-0.937489,-0.010834,0.312488,0.773006,...,1.000000,0.000000,0.153374,0.027607,0.000000,0.052147,0.248466,0.000000,0.226994,adm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347,A220_tri_p_4.csv,0.035306,0.304337,-0.949603,-0.130634,0.317290,-0.936027,-0.050493,0.318514,29.347163,...,0.726950,0.535461,0.294326,0.127660,0.248227,0.166667,0.280142,0.031915,0.209220,tri
348,A220_tri_v_1.csv,0.004303,0.349264,-0.936222,-0.141737,0.344295,-0.927693,-0.073560,0.356190,25.533621,...,1.000000,0.025862,0.396552,0.000000,0.068966,0.422414,0.353448,0.000000,0.025862,tri
349,A220_tri_v_2.csv,-0.009040,0.363339,-0.930700,-0.200038,0.342350,-0.915456,-0.112857,0.365205,40.793168,...,1.000000,0.136646,0.080745,0.136646,0.000000,0.155280,0.161491,0.000000,0.149068,tri
350,A220_tri_v_3.csv,0.035275,0.367004,-0.927336,-0.142579,0.347621,-0.924963,-0.057953,0.368221,40.119463,...,1.000000,0.463087,0.295302,0.281879,0.395973,0.234899,0.228188,0.000000,0.241611,tri


Convert everything to numpy

In [9]:
# Convert your data frame's columns into arrays
x = df_tmp.loc[:, df_tmp.columns.str.startswith(('AU', "gaze", "pose"))].to_numpy()
x

array([[ 0.07177563,  0.31360676, -0.94278772, ...,  0.09340659,
         0.        ,  0.24725275],
       [ 0.04090178,  0.31861908, -0.94408726, ...,  0.02489627,
         0.        ,  0.21991701],
       [ 0.0271765 ,  0.33338437, -0.93937493, ...,  0.2       ,
         0.        ,  0.2893617 ],
       ...,
       [-0.00903987,  0.36333891, -0.93069998, ...,  0.16149068,
         0.        ,  0.14906832],
       [ 0.03527517,  0.36700392, -0.92733556, ...,  0.22818792,
         0.        ,  0.24161074],
       [ 0.03449913,  0.27093415, -0.95425133, ...,  0.37299035,
         0.        ,  0.22508039]])

In [10]:
len(x)

352

In [11]:
y = df_tmp['emotion_1'].to_numpy()
y

array(['adm', 'adm', 'adm', 'adm', 'adm', 'adm', 'adm', 'adm', 'amu',
       'amu', 'amu', 'amu', 'amu', 'amu', 'amu', 'amu', 'ang', 'ang',
       'ang', 'ang', 'ang', 'ang', 'ang', 'ang', 'anx', 'anx', 'anx',
       'anx', 'anx', 'anx', 'anx', 'anx', 'awe', 'awe', 'awe', 'awe',
       'awe', 'awe', 'awe', 'awe', 'bor', 'bor', 'bor', 'bor', 'bor',
       'bor', 'bor', 'bor', 'conc', 'conc', 'conc', 'conc', 'conc',
       'conc', 'conc', 'conc', 'conf', 'conf', 'conf', 'conf', 'conf',
       'conf', 'conf', 'conf', 'cont', 'cont', 'cont', 'cont', 'cont',
       'cont', 'cont', 'cont', 'det', 'det', 'det', 'det', 'det', 'det',
       'det', 'det', 'disa', 'disa', 'disa', 'disa', 'disa', 'disa',
       'disa', 'disa', 'disg', 'disg', 'disg', 'disg', 'disg', 'disg',
       'disg', 'disg', 'dist', 'dist', 'dist', 'dist', 'dist', 'dist',
       'dist', 'dist', 'dou', 'dou', 'dou', 'dou', 'dou', 'dou', 'dou',
       'dou', 'ele', 'ele', 'ele', 'ele', 'ele', 'ele', 'ele', 'ele',
       'emb', 

In [12]:
len(y)

352

Split into training and test sets

In [13]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [14]:
# feed into your classifier 
clf = svm.SVC()
clf.fit(x_train, y_train)

SVC()

Lets look at the predictions...

In [15]:
clf.predict(x_test)

array(['dist', 'dist', 'dist', 'dist', 'dist', 'dist', 'env', 'dist',
       'env', 'dist', 'dist', 'dist', 'env', 'dist', 'dist', 'dist',
       'dist', 'dist', 'env', 'ins', 'dist', 'dist', 'dist', 'dist',
       'dist', 'dist', 'dist', 'dist', 'dist', 'dist', 'ins', 'dist',
       'dist', 'dist', 'dist', 'dist', 'dist', 'ten', 'dist', 'dist',
       'env', 'ten', 'dist', 'dist', 'dist', 'dist', 'env', 'dist',
       'dist', 'ten', 'scha', 'env', 'dist', 'dist', 'ten', 'dist',
       'dist', 'dist', 'dist', 'dist', 'ten', 'dist', 'dist', 'dist',
       'dist', 'dist', 'ten', 'dist', 'env', 'dist', 'dist'], dtype=object)

In [16]:
y_test

array(['emb', 'tri', 'neg_sur', 'neg_sur', 'ple', 'sha', 'neg_sur', 'exc',
       'mov', 'fea', 'gra', 'adm', 'conf', 'anx', 'hop', 'gui', 'gui',
       'rej', 'reg', 'conc', 'ele', 'bor', 'hap', 'tri', 'pri', 'gui',
       'fea', 'awe', 'disa', 'conc', 'sex', 'rej', 'gra', 'emb', 'conc',
       'ele', 'bor', 'sat', 'adm', 'amu', 'conc', 'cont', 'ang', 'dou',
       'nos', 'sha', 'adm', 'neu', 'pos_sur', 'det', 'bor', 'disa', 'sad',
       'sar', 'rel', 'adm', 'ele', 'ang', 'emb', 'sar', 'pea', 'awe',
       'sad', 'disg', 'neu', 'fea', 'det', 'neg_sur', 'conf', 'disa',
       'int'], dtype=object)

Look at the total score, not very good...

In [17]:
clf.score(x_test, y_test)

0.0

Lets try with some other kernels...

In [18]:
# polynomial kernel
clf = svm.SVC(kernel = "poly")
clf.fit(x_train, y_train)
clf.score(x_test, y_test)

0.0

In [19]:
# rbf
clf = svm.SVC(kernel = "rbf")
clf.fit(x_train, y_train)
clf.score(x_test, y_test)

0.0

In [20]:
# linear
clf = svm.SVC(kernel = "linear")
clf.fit(x_train, y_train)
clf.score(x_test, y_test)

0.36619718309859156

In [21]:
clf.predict(x_test)

array(['scha', 'tri', 'ang', 'neg_sur', 'ele', 'gui', 'disa', 'sar',
       'nos', 'sad', 'tri', 'nos', 'conf', 'anx', 'hop', 'disa', 'disa',
       'reg', 'reg', 'conf', 'mov', 'rel', 'hap', 'adm', 'pri', 'gui',
       'anx', 'awe', 'rej', 'conc', 'sex', 'disa', 'hop', 'hop', 'cont',
       'ins', 'det', 'ple', 'adm', 'amu', 'conf', 'cont', 'ang', 'nos',
       'hop', 'fea', 'nos', 'rel', 'exc', 'det', 'ins', 'disa', 'anx',
       'sar', 'bor', 'nos', 'ele', 'disg', 'bor', 'sat', 'bor', 'amu',
       'sad', 'sad', 'dou', 'fea', 'det', 'neg_sur', 'sad', 'disa', 'int'],
      dtype=object)

In [22]:
y_test

array(['emb', 'tri', 'neg_sur', 'neg_sur', 'ple', 'sha', 'neg_sur', 'exc',
       'mov', 'fea', 'gra', 'adm', 'conf', 'anx', 'hop', 'gui', 'gui',
       'rej', 'reg', 'conc', 'ele', 'bor', 'hap', 'tri', 'pri', 'gui',
       'fea', 'awe', 'disa', 'conc', 'sex', 'rej', 'gra', 'emb', 'conc',
       'ele', 'bor', 'sat', 'adm', 'amu', 'conc', 'cont', 'ang', 'dou',
       'nos', 'sha', 'adm', 'neu', 'pos_sur', 'det', 'bor', 'disa', 'sad',
       'sar', 'rel', 'adm', 'ele', 'ang', 'emb', 'sar', 'pea', 'awe',
       'sad', 'disg', 'neu', 'fea', 'det', 'neg_sur', 'conf', 'disa',
       'int'], dtype=object)

In [23]:
# sigmoid
clf = svm.SVC(kernel = "sigmoid")
clf.fit(x_train, y_train)
clf.score(x_test, y_test)

0.0